# Using Apache Iceberg with Spark 3 in CML

The official documentation for Apache Iceberg with Spark is located at [this link](https://iceberg.apache.org/#getting-started/#using-iceberg-in-spark-3)

For a full list of Apache Iceberg terms, please visit [this link](https://iceberg.apache.org/#terms/)

### Start a PySpark Session as shown below. You will want to set the Spark Catalog configurations as shown

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
  .config("spark.jars.packages","org.apache.iceberg:iceberg-spark3-runtime:0.12.1") \
  .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog") \
  .config("spark.sql.catalog.spark_catalog.type","hive") \
  .getOrCreate()

### Iceberg comes with catalogs that enable SQL commands to manage tables and load them by name. 
### Catalogs are configured using properties under spark.sql.catalog.(catalog_name).

In [2]:
  # Using a local Spark Catalog

spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.testdb ")
spark.sql("USE spark_catalog.testdb")
spark.sql("SHOW CURRENT NAMESPACE").show()
#spark.sql("DROP TABLE testtable")

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|   testdb|
+-------------+---------+



### You can use simple Spark SQL commands to create Spark tables as you always have. Just make sure to specify the USING iceberg clause.

In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS testtable (id bigint, data string) USING iceberg")

DataFrame[]

### To select a specific table snapshot or the snapshot at some time, Iceberg supports two Spark read options:

* snapshot-id selects a specific table snapshot
* as-of-timestamp selects the current snapshot at a timestamp, in milliseconds

#### You can view all snapshots associated with the table

In [4]:
spark.read.format("iceberg").load("spark_catalog.testdb.testtable.snapshots").show(20, False)

+-----------------------+-------------------+-------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                                       |summary                                                                                                                                                                                                                                                             

#### Or a full table version history 

In [5]:
spark.read.format("iceberg").load("spark_catalog.testdb.testtable.history").show(20, False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2021-11-19 10:08:15.886|3492322305412256778|null               |true               |
|2021-11-19 10:21:55.566|107550928418073495 |3492322305412256778|true               |
|2021-11-19 10:28:30.292|1545408577715836429|107550928418073495 |true               |
|2021-11-19 10:28:33.014|2649908378426818566|1545408577715836429|true               |
|2021-11-19 10:34:33.135|3533974690975069464|2649908378426818566|true               |
|2021-11-20 00:32:49.144|3355558037703162022|3533974690975069464|true               |
|2021-11-20 00:33:28.398|3383995530425746467|3355558037703162022|true               |
|2021-11-20 01:15:01.931|4072529096957210655|3383995530425746467|true               |
+-----------------------+-------------------+---------

#### To show a table’s data files and each file’s metadata, run:

In [6]:
spark.read.format("iceberg").load("spark_catalog.testdb.testtable.files").show(20, False)

+-------+---------------------------------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------------+----------------+-----------------+----------------+-----------------------+-----------------------+------------+-------------+------------+
|content|file_path                                                                                                                              |file_format|record_count|file_size_in_bytes|column_sizes      |value_counts    |null_value_counts|nan_value_counts|lower_bounds           |upper_bounds           |key_metadata|split_offsets|equality_ids|
+-------+---------------------------------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------------+----------------+-----------------+----------------+-----------------------+------------------

### A manifest file is a metadata file that lists a subset of data files that make up a snapshot.

### Each data file in a manifest is stored with a partition tuple, column-level stats, and summary information used to prune splits during scan planning.

#### To show a table’s file manifests and each file’s metadata, run:

In [7]:
spark.read.format("iceberg").load("spark_catalog.testdb.testtable.manifests").show(20, False)

+----------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+-------------------+
|path                                                                                                                        |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|partition_summaries|
+----------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+-------------------+
|s3a://gd01-uat2/warehouse/tablespace/external/hive/testdb.db/testtable/metadata/30abaa14-3bd0-4b4f-bd9b-65431d40d90d-m0.avro|5650  |0                |4072529096957210655|2                     |0       

## Time Travel

### Using snapshots as shown above, we can insert some data into the table and roll back to its original state

In [8]:
# Insert using Iceberg format
spark.sql("INSERT INTO testtable VALUES (1, 'x'), (2, 'y'), (3, 'z')")

DataFrame[]

In [9]:
# Query using select
spark.sql("SELECT * FROM testtable").show()

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   d|
|  2|   e|
|  3|   f|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
+---+----+
only showing top 20 rows



In [10]:
# Query using DF - All Data
df = spark.table("spark_catalog.testdb.testtable")
df.show(100)

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   d|
|  2|   e|
|  3|   f|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   d|
|  2|   e|
|  3|   f|
+---+----+



In [11]:
from datetime import datetime

# current date and time
now = datetime.now()

timestamp = datetime.timestamp(now)
print("timestamp =", timestamp)

timestamp = 1638410067.273329


#### Timestamps can be tricky. Please make sure to round your timestamp as shown below.

In [12]:
# Query using a point in time
df = spark.read.option("as-of-timestamp", int(timestamp*1000)).format("iceberg").load("spark_catalog.testdb.testtable")
df.show(100)

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   d|
|  2|   e|
|  3|   f|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   a|
|  2|   b|
|  3|   c|
|  1|   x|
|  2|   y|
|  3|   z|
|  1|   d|
|  2|   e|
|  3|   f|
+---+----+



In [67]:
# Insert using Iceberg format
spark.sql("INSERT INTO testtable VALUES (1, 'd'), (2, 'e'), (3, 'f')")

DataFrame[]